In [1]:
import pandas as pd
from conf.users import Users
import conf.figures as figures
from conf.actions import get_name
from extract_distance import KEY_ACTION, KEY_ID
import matplotlib.pyplot as plt
import sys, os


users = Users(pretest=True)

In [2]:
COLOR_PICK = [
        (0, 0, 0),\
        (0, 1, 0),\
        (1, 0, 0),\
        (0, 0, 1)]

COLOR_PLACE = [
        (0, 0, 0),\
        (0, 1, 0),\
        (1, 0, 0),\
        (0, 0, 1)]

LEGEND_PICK = [ "Block", "Zone", "Color", "Shape"]

LEGEND_PLACE = [ "Place", "Expand 1", "Expand 2", "Expand 4"]

KEY_PICK = ["block_to_pick", "stock_to_pick", "color_to_pick", "shape_to_pick"]
KEY_PLACE = ["pos_to_place", "expand_1", "expand_2", "expand_4"]

IDX_ZONE = [0,1,2,3]
IDX_EXPAND = [0,1,2,3]

def get_distances(user, figure, act, id_event, key):
    csvfile = ("../data/%s/%s/events/%s/%d_table.csv" % (user, figure, act, id_event))
    distances = {}
    df = pd.DataFrame(data=pd.read_csv (csvfile))
    for idx in df.index:
        distances[df.at[idx, "timestamp"]] = df.at[idx, key]
    return distances
                
def scatter(ax, data, color, label):
    x = []
    y = []
    for key in data.keys():
        x.append(key)
        y.append(data[key])
    ax.scatter(x,y,color=color,label=label)
    
def plot_zone(user, figures, pick_dir):
    for figId in figures.get_id_list():
        dist = []
        figure = figures.get_name(figId)
        try:
            labels_csv = ("../data/%s/%s/events/pick/labels.csv" % (user, figure))
            labels = pd.DataFrame(data=pd.read_csv (labels_csv))
            for idx in labels.index:
                tmp = {}
                for zone in IDX_ZONE:
                    d = get_distances(user, figure, "pick", labels.at[idx, KEY_ID], KEY_PICK[zone])
                    tmp[zone] = d
                dist.append(tmp)
        except FileNotFoundError as e:
            continue
        idx=0
        for data in dist:
            fig, ax = plt.subplots(1, 1, figsize=(5, 5))
            ax.set_xlabel('Time (ms)')
            ax.set_ylabel('Distance (mm)')
            for zone in IDX_ZONE:
                scatter(ax, data[zone], COLOR_PICK[zone], LEGEND_PICK[zone])
            ax.legend()
            ax.grid(True)
            fig.savefig("%s/zone_%s_%d.png" % (pick_dir, figure, idx))
            plt.close(fig)
            idx += 1
            
def plot_expand(user, figures, place_dir):
    for figId in figures.get_id_list():
        dist = []
        figure = figures.get_name(figId)
        try:
            labels_csv = ("../data/%s/%s/events/place/labels.csv" % (user, figure))
            labels = pd.DataFrame(data=pd.read_csv (labels_csv))
            for idx in labels.index:
                tmp = {}
                for expand in IDX_EXPAND:
                    d = get_distances(user, figure, "place", labels.at[idx, KEY_ID], KEY_PLACE[expand])
                    tmp[expand] = d
                dist.append(tmp)
        except FileNotFoundError as e:
            continue
        idx=0
        for data in dist:
            fig, ax = plt.subplots(1, 1, figsize=(5, 5))
            ax.set_xlabel('Time (ms)')
            ax.set_ylabel('Distance (mm)')
            for expand in IDX_EXPAND:
                scatter(ax, data[expand], COLOR_PLACE[expand], LEGEND_PLACE[expand])
            ax.legend()
            ax.grid(True)
            fig.savefig("%s/expand_%s_%d.png" % (place_dir, figure, idx))
            plt.close(fig)
            idx += 1
            
for user in users.get_users_id_list():
    user_dir = "output/png/%s" % user
    pick_dir = "output/png/%s/pick" % user
    place_dir = "output/png/%s/place" % user
    if(not os.path.exists(user_dir)):
        os.mkdir(user_dir)
        os.mkdir(pick_dir)
        os.mkdir(place_dir)
    plot_zone(user, figures, pick_dir)
    plot_expand(user, figures, place_dir)
    